In [45]:
# %load Webscraper.py
import pickle
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from time import sleep
import requests
import re

with open('album_list.pkl', 'rb') as f:
    albums = pickle.load(f)

serach_field = '/html/body/div[1]/search-form/form/input'
click_button = '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[2]/div[5]/search-result-section/div/div[2]/vertical-search-result-items/div/div/vertical-search-result-item/div'

def start_driver():

    options = webdriver.ChromeOptions()
#     options.binary_location = '/opt/google/chrome/google-chrome'
#     options.add_argument('headless')
    driver = webdriver.Chrome(chrome_options=options)

    return driver


def search_and_click(item, serach_field=serach_field, click_button=click_button):
    search = driver.find_element_by_xpath(serach_field)
    search.send_keys(item)
    search.submit()
    go = driver.find_element_by_xpath(click_button)
    go.click()


def lyric_getter(albums, start_ind=0, song_id=0):

    driver = start_driver()
    driver.get('https://genius.com/')
    
    missed_albums = []

    for counter, album in enumerate(albums):
        
        
        try:
            search = driver.find_element_by_xpath('/html/body/div[1]/search-form/form/input')
            search.send_keys(album)
            search.submit()

            sleep(1)

            go = driver.find_element_by_xpath('/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[2]/div[5]/search-result-section/div/div[2]/vertical-search-result-items/div/div/vertical-search-result-item/div')
            go.click()

            sleep(1)

            html            = driver.page_source
            album_soup      = BeautifulSoup(html, 'lxml')

            track_counts    = album_soup.find_all('div', {'class': 'chart_row-number_container chart_row-number_container--align_left'})
            date_released   = album_soup.find('div', {'class': 'metadata_unit'}).text.replace('\n', '').strip()[9:]
            track_listing   = album_soup.find_all('div', {'class': 'chart_row-content'})
            album           = album_soup.find('h1').text

            for count, song in enumerate(track_listing):

                track_count = track_counts[count].find('span').text.replace('\n', '')
                song_title  = track_listing[count].find('h3').text.strip().split('\n')[0]
                href        = track_listing[count].find('a')['href']

                results    = requests.get(href)
                song_soup  = BeautifulSoup(results.content, 'lxml')
                artist     = song_soup.find('a', {'class', 'header_with_cover_art-primary_info-primary_artist'}).text
                lyrics     = re.sub('(\\n){2}', '\\n', song_soup.find('p').text).replace('\n', ' n ')

                song_info = [
                    str(song_id),
                    song_title,
                    href, track_count,
                    date_released,
                    album,
                    lyrics,
                    artist,
                ]


                with open('songs2.psv', 'a+') as f:
                    print('|'.join(song_info), file=f)

                song_id = song_id + 1
                print(counter, album, 'done')

        except:
            missed_albums.append(album)
            
    with open('missed_albums.pkl', 'wb+') as f:
        pickle.dump(missed_albums, f)

    driver.close()

In [26]:
lyric_getter(albums)

KeyboardInterrupt: 

In [21]:
with open('missed_albums.pkl', 'rb') as f:
    missed_albums = pickle.load(f)
    

In [23]:
len(missed_albums)

661

In [24]:
missed_albums

['The Bootleg Series Vol. 7: No Direction Home: the Soundtrack',
 'The Bootleg Series Vol. 9: The Witmark Demos: 1962',
 'The Bootleg Series Vol. 6: Bob Dylan Live 1964',
 'Torch of the Mystics',
 'Just Another Diamond Day',
 'Sweetheart of the Rodeo',
 'Sung Tongs',
 'You Are Free',
 'Central Reservation',
 'Lost in the Dream',
 'Helplessness Blues',
 'Either/Or',
 'Songs from a Room',
 'What We Did on Our Holidays',
 'Crack',
 'The Bootleg Series Vol. 10: Another Self Portrait (1969',
 'Lie Down in the Light',
 'Sun Giant',
 'The Cloud of Unknowing',
 'Joanna Newsom and the Ys Street Band',
 'Roots & Crowns',
 'Yellow House',
 'Behold The Spirit',
 'Dark Was The Night',
 'The Bootleg Series Vol. 8: Tell Tale Signs',
 "The Shepherd's Dog",
 'Our Endless Numbered Days',
 'Quicksand / Cradlesnakes',
 'Everything Is Good Here / Please Come Home',
 'Parallelograms',
 'Divers',
 'Dream River',
 'Furr',
 'Friend',
 'Wild Mountain Nation',
 'We Shall Overcome: The Seeger Sessions',
 'In the 

In [33]:
for count, album in enumerate(missed_albums):
    if count % 10 == 0:
        print(count, album)

0 The Bootleg Series Vol. 7: No Direction Home: the Soundtrack
10 Love's Crushing Diamond
20 She's the Dutchess, He's the Duke
30 Overboard & Down
40 Eyes on the Lines
50 Life Is People
60 We Are Him
70 Axxess & Ace
80 Love Is The Plan, The Plan Is Death
90 Blood Bank EP
100 House and Land
110 The Double EP: The Sea of Split Peas
120 Epic
130 In Our Nature
140 Follow Them True
150 Ghosts Go Blind
160 Because I Was in Love
170 The Optimist LP
180 Someday, The Moon Will Be Gold
190 All Delighted People
200 Bedouine
210 Strong Feelings
220 Fields
230 Emotionalism
240 Short Movie
250 Death Seat
260 Micah P. Hinson and the Opera Circuit
270 Alasdair Roberts
280 Gentle Stream
290 Conor Oberst
300 Eric Bachmann
310 Blood Speaks
320 The Second Gleam EP
330 Don't Fall In Love With Everyone You See
340 Colin Meloy Sings Shirley Collins
350 I Don't Want To Let You Down
360 A Monument
370 By the Way, I Forgive You
380 Luminous Night
390 Year of the Rabbit
400 Beachcomber's Windowsill
410 Mermaid A

In [30]:
lyric_getter(missed_albums, song_id=678)


0 Albums Done
10 Albums Done
20 Albums Done
30 Albums Done
40 Albums Done
50 Albums Done
60 Albums Done
70 Albums Done
80 Albums Done
90 Albums Done
100 Albums Done
110 Albums Done
120 Albums Done
130 Albums Done
140 Albums Done
150 Albums Done
160 Albums Done
170 Albums Done
180 Albums Done
190 Albums Done
200 Albums Done
210 Albums Done
220 Albums Done
230 Albums Done
240 Albums Done
250 Albums Done
260 Albums Done
270 Albums Done
280 Albums Done
290 Albums Done
300 Albums Done
310 Albums Done
320 Albums Done
330 Albums Done
340 Albums Done
350 Albums Done
360 Albums Done
370 Albums Done
380 Albums Done
390 Albums Done
400 Albums Done
410 Albums Done
420 Albums Done
430 Albums Done
440 Albums Done
450 Albums Done
460 Albums Done
470 Albums Done
480 Albums Done
490 Albums Done
500 Albums Done
510 Albums Done
520 Albums Done
530 Albums Done
540 Albums Done
550 Albums Done
560 Albums Done
570 Albums Done
580 Albums Done
590 Albums Done
600 Albums Done
610 Albums Done
620 Albums Done
630

In [31]:
with open('missed_albums.pkl', 'rb') as f:
    missed_albums = pickle.load(f)


In [32]:
len(missed_albums)

596

In [34]:
lyric_getter(missed_albums, song_id=1632)

0 Albums Done
10 Albums Done
20 Albums Done
30 Albums Done
40 Albums Done
50 Albums Done
60 Albums Done
70 Albums Done
80 Albums Done
90 Albums Done
100 Albums Done
110 Albums Done
120 Albums Done
130 Albums Done
140 Albums Done
150 Albums Done
160 Albums Done
170 Albums Done
180 Albums Done
190 Albums Done
200 Albums Done
210 Albums Done
220 Albums Done
230 Albums Done
240 Albums Done
250 Albums Done
260 Albums Done
270 Albums Done
280 Albums Done
290 Albums Done
300 Albums Done
310 Albums Done
320 Albums Done
330 Albums Done
340 Albums Done
350 Albums Done
360 Albums Done
370 Albums Done
380 Albums Done
390 Albums Done
400 Albums Done
410 Albums Done
420 Albums Done
430 Albums Done
440 Albums Done
450 Albums Done
460 Albums Done
470 Albums Done
480 Albums Done
490 Albums Done
500 Albums Done
510 Albums Done
520 Albums Done
530 Albums Done
540 Albums Done
550 Albums Done
560 Albums Done
570 Albums Done
580 Albums Done
590 Albums Done


In [39]:
with open('missed_albums.pkl', 'rb') as f:
    missed_albums = pickle.load(f)

In [40]:
len(missed_albums)

426

In [38]:
missed_albums

['The Bootleg Series Vol. 7: No Direction Home: the Soundtrack',
 'Torch of the Mystics',
 'Helplessness Blues',
 'The Bootleg Series Vol. 10: Another Self Portrait (1969',
 'The Cloud of Unknowing',
 'Joanna Newsom and the Ys Street Band',
 'Behold The Spirit',
 "Love's Crushing Diamond",
 "WIT'S END",
 'Light Upon the Lake',
 'Singing Saw',
 'Litany of Echoes',
 'Ghosts of the Great Highway',
 "She's the Dutchess, He's the Duke",
 'Burning the Threshold',
 'What Am I Going to Do With Everything I Know',
 'Sometimes I Wish We Were An Eagle',
 'March of the Zapotec/Holland EP',
 'Seven Swans',
 'The Creek Drank The Cradle',
 'Castaways and Cutouts',
 'The Weather Station',
 'Peasant',
 'Colter Wall',
 'Lodestar',
 'Modern Country',
 'All Are Saved',
 'The Offer',
 'Way Out Weather',
 'Double Exposure',
 'Haw',
 'Half Way Home',
 'Love Will Prevail',
 'Life Is People',
 'Silent Hour / Golden Mile',
 'Apocalypse',
 'Innundir Skinni',
 'Rockwell',
 'To Be Still',
 'Rook',
 'Pride',
 'We A

In [41]:
import pandas as pd

In [44]:
pd.concat((pd.read_csv('songs.psv', sep='|', header=None), pd.read_csv('songs1.psv', sep='|', header=None)))

,0,1,2,3,4,5,6,7
0,0,A Minor Place,https://genius.com/Bonnie-prince-billy-a-minor...,1.0,1999,I See a Darkness,[hook] n I've been to a minor place n And I ca...,"Bonnie ""Prince"" Billy"
1,1,Nomadic Revery (All Around),https://genius.com/Bonnie-prince-billy-nomadic...,2.0,1999,I See a Darkness,"Today was one where, lost in thought n I reall...","Bonnie ""Prince"" Billy"
2,2,I See a Darkness,https://genius.com/Bonnie-prince-billy-i-see-a...,3.0,1999,I See a Darkness,Well you're my friend n (That's what you told ...,"Bonnie ""Prince"" Billy"
3,3,Another Day Full of Dread,https://genius.com/Bonnie-prince-billy-another...,4.0,1999,I See a Darkness,I like to have a good time n Any of my friends...,"Bonnie ""Prince"" Billy"
4,4,Death to Everyone,https://genius.com/Bonnie-prince-billy-death-t...,5.0,1999,I See a Darkness,"I am here, right here n Where God puts non asu...","Bonnie ""Prince"" Billy"
5,5,Knockturne,https://genius.com/Bonnie-prince-billy-knocktu...,6.0,1999,I See a Darkness,Fire-burned and blew out flowers n Showing me ...,"Bonnie ""Prince"" Billy"
6,6,Madeleine Mary,https://genius.com/Bonnie-prince-billy-madelei...,7.0,1999,I See a Darkness,Sing a song of madeleine-mary n A tune that al...,"Bonnie ""Prince"" Billy"
7,7,Song for the New Breed,https://genius.com/Bonnie-prince-billy-song-fo...,8.0,1999,I See a Darkness,Your little feet n Your sharp teeth n The way ...,"Bonnie ""Prince"" Billy"
8,8,Today I Was an Evil One,https://genius.com/Bonnie-prince-billy-today-i...,9.0,1999,I See a Darkness,Tonight my eyes were hurting much n As I had s...,"Bonnie ""Prince"" Billy"
9,9,Black,https://genius.com/Bonnie-prince-billy-black-l...,10.0,1999,I See a Darkness,"Black, you are my enemy n And I cannot get clo...","Bonnie ""Prince"" Billy"


In [48]:
lyric_getter(missed_albums, song_id=3103)

2 Helplessness Blues done
2 Helplessness Blues done
2 Helplessness Blues done
2 Helplessness Blues done
2 Helplessness Blues done
2 Helplessness Blues done
2 Helplessness Blues done
2 Helplessness Blues done
2 Helplessness Blues done
2 Helplessness Blues done
2 Helplessness Blues done
2 Helplessness Blues done
7 Love's Crushing Diamond done
7 Love's Crushing Diamond done
7 Love's Crushing Diamond done
7 Love's Crushing Diamond done
7 Love's Crushing Diamond done
7 Love's Crushing Diamond done
7 Love's Crushing Diamond done
7 Love's Crushing Diamond done
8 Wit's End done
8 Wit's End done
8 Wit's End done
8 Wit's End done
8 Wit's End done
8 Wit's End done
8 Wit's End done
8 Wit's End done
9 Light Upon the Lake done
9 Light Upon the Lake done
9 Light Upon the Lake done
9 Light Upon the Lake done
9 Light Upon the Lake done
9 Light Upon the Lake done
9 Light Upon the Lake done
9 Light Upon the Lake done
9 Light Upon the Lake done
9 Light Upon the Lake done
10 Singing Saw done
10 Singing Saw

41 We Are Him done
41 We Are Him done
42 Akron/Family done
42 Akron/Family done
42 Akron/Family done
42 Akron/Family done
42 Akron/Family done
42 Akron/Family done
42 Akron/Family done
42 Akron/Family done
42 Akron/Family done
42 Akron/Family done
42 Akron/Family done
42 Akron/Family done
42 Akron/Family done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
44 Niño Rojo done
47 The Grotto done
47 The Grotto done
47 The Grotto done
47 The Grotto done
47 The Grotto done
47 The Grotto done
47 The Grotto done
47 The Grotto done
47 The Grotto done
47 The Grotto done
48 Field Songs done
48 Field Songs done
48 Field Songs done
48 Field Songs done
48 Field Songs done
48 Field Songs done
48 Field Songs done
48 Field Songs done
48 Field Songs done
48 Field Songs done
48 Field So

89 The Saga of Mayflower May done
89 The Saga of Mayflower May done
89 The Saga of Mayflower May done
89 The Saga of Mayflower May done
89 The Saga of Mayflower May done
89 The Saga of Mayflower May done
89 The Saga of Mayflower May done
89 The Saga of Mayflower May done
89 The Saga of Mayflower May done
89 The Saga of Mayflower May done
89 The Saga of Mayflower May done
90 Oh Me Oh My...The Way the Day Goes By the Sun Is Setting Dogs Are Dreaming Lovesongs of the Christmas Spirit done
90 Oh Me Oh My...The Way the Day Goes By the Sun Is Setting Dogs Are Dreaming Lovesongs of the Christmas Spirit done
90 Oh Me Oh My...The Way the Day Goes By the Sun Is Setting Dogs Are Dreaming Lovesongs of the Christmas Spirit done
90 Oh Me Oh My...The Way the Day Goes By the Sun Is Setting Dogs Are Dreaming Lovesongs of the Christmas Spirit done
90 Oh Me Oh My...The Way the Day Goes By the Sun Is Setting Dogs Are Dreaming Lovesongs of the Christmas Spirit done
90 Oh Me Oh My...The Way the Day Goes By 

118 Tyranny done
118 Tyranny done
118 Tyranny done
118 Tyranny done
118 Tyranny done
122 Altüst done
122 Altüst done
122 Altüst done
122 Altüst done
122 Altüst done
126 Lateness of Dancers done
126 Lateness of Dancers done
126 Lateness of Dancers done
126 Lateness of Dancers done
126 Lateness of Dancers done
126 Lateness of Dancers done
126 Lateness of Dancers done
126 Lateness of Dancers done
126 Lateness of Dancers done
126 Lateness of Dancers done
128 Brothers and Sisters of the Eternal Son done
128 Brothers and Sisters of the Eternal Son done
128 Brothers and Sisters of the Eternal Son done
128 Brothers and Sisters of the Eternal Son done
128 Brothers and Sisters of the Eternal Son done
128 Brothers and Sisters of the Eternal Son done
128 Brothers and Sisters of the Eternal Son done
128 Brothers and Sisters of the Eternal Son done
128 Brothers and Sisters of the Eternal Son done
128 Brothers and Sisters of the Eternal Son done
128 Brothers and Sisters of the Eternal Son done
130 A 

156 July Flame done
156 July Flame done
156 July Flame done
157 Yankee Reality done
157 Yankee Reality done
157 Yankee Reality done
157 Yankee Reality done
157 Yankee Reality done
157 Yankee Reality done
157 Yankee Reality done
157 Yankee Reality done
157 Yankee Reality done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
158 Un Día en Suburbia done
159 Miles Benjamin Anthony Robinson done
159 Miles Benjamin Anthony Robinson done
159 Miles Benjamin Anthony Robinson done
159 Miles Benjamin Anthony Robinson done
159 Miles Benjamin Anthony Robinson done
159 Miles Benjamin Anthony Robinson done
15

...

262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in My Coffee done
262 Clouds in 

298 Creep On Creepin' On done
298 Creep On Creepin' On done
298 Creep On Creepin' On done
298 Creep On Creepin' On done
298 Creep On Creepin' On done
298 Creep On Creepin' On done
298 Creep On Creepin' On done
298 Creep On Creepin' On done
298 Creep On Creepin' On done
301 Through the Devil Softly done
301 Through the Devil Softly done
301 Through the Devil Softly done
301 Through the Devil Softly done
301 Through the Devil Softly done
301 Through the Devil Softly done
301 Through the Devil Softly done
301 Through the Devil Softly done
301 Through the Devil Softly done
301 Through the Devil Softly done
301 Through the Devil Softly done
302 Elvis Perkins in Dearland done
302 Elvis Perkins in Dearland done
302 Elvis Perkins in Dearland done
302 Elvis Perkins in Dearland done
302 Elvis Perkins in Dearland done
302 Elvis Perkins in Dearland done
302 Elvis Perkins in Dearland done
302 Elvis Perkins in Dearland done
302 Elvis Perkins in Dearland done
302 Elvis Perkins in Dearland done
304 Ev

333 The Ghost Who Walks   done
333 The Ghost Who Walks   done
333 The Ghost Who Walks   done
333 The Ghost Who Walks   done
333 The Ghost Who Walks   done
333 The Ghost Who Walks   done
333 The Ghost Who Walks   done
333 The Ghost Who Walks   done
333 The Ghost Who Walks   done
333 The Ghost Who Walks   done
333 The Ghost Who Walks   done
334 Sunday At Devil Dirt done
334 Sunday At Devil Dirt done
334 Sunday At Devil Dirt done
334 Sunday At Devil Dirt done
334 Sunday At Devil Dirt done
334 Sunday At Devil Dirt done
334 Sunday At Devil Dirt done
334 Sunday At Devil Dirt done
334 Sunday At Devil Dirt done
334 Sunday At Devil Dirt done
334 Sunday At Devil Dirt done
334 Sunday At Devil Dirt done
335 Fisherman's Woman done
335 Fisherman's Woman done
335 Fisherman's Woman done
335 Fisherman's Woman done
335 Fisherman's Woman done
335 Fisherman's Woman done
335 Fisherman's Woman done
335 Fisherman's Woman done
335 Fisherman's Woman done
335 Fisherman's Woman done
335 Fisherman's Woman done
33

365 The Hazards of Love done
365 The Hazards of Love done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
366 The Historical Conquests of Josh Ritter done
368 Hawks And Doves done
368 Hawks And Doves done
368 Hawks 

400 Tales From Terra Firma done
400 Tales From Terra Firma done
400 Tales From Terra Firma done
400 Tales From Terra Firma done
400 Tales From Terra Firma done
400 Tales From Terra Firma done
400 Tales From Terra Firma done
400 Tales From Terra Firma done
400 Tales From Terra Firma done
401 The Gathering done
401 The Gathering done
401 The Gathering done
401 The Gathering done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
402 Outer South done
404 How I Loved You done
404 How I Loved You done
404 How I Loved You done
404 How I Loved You done
404 How I Loved You done
404 How I Loved You done
404 How I Loved You done
404 How I Loved You done
404 How I Loved You done
404 How I Loved You done
406 Saltbreakers done
406 Saltbrea

In [49]:
missed_albums

['The Bootleg Series Vol. 7: No Direction Home: the Soundtrack',
 'Torch of the Mystics',
 'Helplessness Blues',
 'The Bootleg Series Vol. 10: Another Self Portrait (1969',
 'The Cloud of Unknowing',
 'Joanna Newsom and the Ys Street Band',
 'Behold The Spirit',
 "Love's Crushing Diamond",
 "WIT'S END",
 'Light Upon the Lake',
 'Singing Saw',
 'Litany of Echoes',
 'Ghosts of the Great Highway',
 "She's the Dutchess, He's the Duke",
 'Burning the Threshold',
 'What Am I Going to Do With Everything I Know',
 'Sometimes I Wish We Were An Eagle',
 'March of the Zapotec/Holland EP',
 'Seven Swans',
 'The Creek Drank The Cradle',
 'Castaways and Cutouts',
 'The Weather Station',
 'Peasant',
 'Colter Wall',
 'Lodestar',
 'Modern Country',
 'All Are Saved',
 'The Offer',
 'Way Out Weather',
 'Double Exposure',
 'Haw',
 'Half Way Home',
 'Love Will Prevail',
 'Life Is People',
 'Silent Hour / Golden Mile',
 'Apocalypse',
 'Innundir Skinni',
 'Rockwell',
 'To Be Still',
 'Rook',
 'Pride',
 'We A

In [50]:
with open('missed_albums.pkl', 'rb') as f:
    missed_albums = pickle.load(f)

In [51]:
len(missed_albums)

137

In [52]:
missed_albums

['The Bootleg Series Vol. 7: No Direction Home: the Soundtrack',
 'Torch of the Mystics',
 'Helplessness Blues',
 'The Bootleg Series Vol. 10: Another Self Portrait (1969',
 'The Cloud of Unknowing',
 'Joanna Newsom and the Ys Street Band',
 'Behold The Spirit',
 'Light Upon the Lake',
 'Singing Saw',
 'Litany of Echoes',
 "She's the Dutchess, He's the Duke",
 'Burning the Threshold',
 'What Am I Going to Do With Everything I Know',
 'March of the Zapotec/Holland EP',
 'Lodestar',
 'Way Out Weather',
 'Love Will Prevail',
 'Love Songs for Patriots',
 'The Milk',
 'Ballads of Living and Dying',
 'Summoning Suns',
 'With Light and With Love',
 'Where Shine New Lights',
 'Love Is The Plan, The Plan Is Death',
 'LNDN DRGS',
 'Southland Mission',
 'Shrink Dust',
 'Small Town Heroes',
 'Cervantine',
 'Belly of the Lion',
 '(a)spera',
 'The Living and The Dead',
 'Follow Them True',
 'Pleasure',
 'Semper Femina',
 'I Am All Your Own',
 'Live At The Cellar Door',
 'My Garden State',
 'The Orna

In [61]:
def lyric_getter1(albums, start_ind=0, song_id=0):

    driver = start_driver()
    driver.get('https://genius.com/')
    
    missed_albums = []

    for counter, album in enumerate(albums):
        
        
        try:
            search = driver.find_element_by_xpath('/html/body/div[1]/search-form/form/input')
            search.send_keys(album)
            search.submit()

            sleep(2)

            go = driver.find_element_by_xpath('/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[2]/div[5]/search-result-section/div/div[2]/vertical-search-result-items/div/div/vertical-search-result-item/div')
            go.click()

            sleep(2)

            html            = driver.page_source
            album_soup      = BeautifulSoup(html, 'lxml')

            track_counts    = album_soup.find_all('div', {'class': 'chart_row-number_container chart_row-number_container--align_left'})
            date_released   = album_soup.find('div', {'class': 'metadata_unit'}).text.replace('\n', '').strip()[9:]
            track_listing   = album_soup.find_all('div', {'class': 'chart_row-content'})
            album           = album_soup.find('h1').text

            for count, song in enumerate(track_listing):

                track_count = track_counts[count].find('span').text.replace('\n', '')
                song_title  = track_listing[count].find('h3').text.strip().split('\n')[0]
                href        = track_listing[count].find('a')['href']

                results    = requests.get(href)
                song_soup  = BeautifulSoup(results.content, 'lxml')
                artist     = song_soup.find('a', {'class', 'header_with_cover_art-primary_info-primary_artist'}).text
                lyrics     = re.sub('(\\n){2}', '\\n', song_soup.find('p').text).replace('\n', ' n ')

                song_info = [
                    str(song_id),
                    song_title,
                    href, track_count,
                    date_released,
                    album,
                    lyrics,
                    artist,
                ]


                with open('songs3.psv', 'a+') as f:
                    print('|'.join(song_info), file=f)

                song_id = song_id + 1
            if counter % 10 == 0:
                print(counter, album, 'done')

        except:
            missed_albums.append(album)
            print(counter, album, 'Missed')
            
    with open('missed_albums.pkl', 'wb+') as f:
        pickle.dump(missed_albums, f)

    driver.close()

In [62]:
len(missed_albums)

137

In [63]:
lyric_getter1(missed_albums, song_id=6507)

0 The Bootleg Series Vol. 7: No Direction Home: the Soundtrack Missed
1 Torch of the Mystics Missed
2 Helplessness Blues Missed
3 The Bootleg Series Vol. 10: Another Self Portrait (1969 Missed
4 The Cloud of Unknowing Missed
5 Joanna Newsom and the Ys Street Band Missed
6 Behold The Spirit Missed
7 Light Upon the Lake Missed
8 Singing Saw Missed
9 Litany of Echoes Missed
10 She's the Dutchess, He's the Duke Missed
11 Burning the Threshold Missed
12 What Am I Going to Do With Everything I Know Missed
13 March of the Zapotec/Holland EP Missed
14 Lodestar Missed
15 Way Out Weather Missed
16 Love Will Prevail Missed
17 Love Songs for Patriots Missed
20 Summoning Suns Missed
22 Where Shine New Lights Missed
23 Love Is The Plan, The Plan Is Death Missed
24 LNDN DRGS Missed
25 Southland Mission Missed
29 Belly of the Lion Missed
30 (a)spera Missed
32 Follow Them True Missed
35 I Am All Your Own Missed
36 Live At The Cellar Door Missed
37 My Garden State Missed
40 In the Cool of the Day done
5

In [65]:
df = pd.read_csv('list_of_albums.csv')

In [103]:
df.loc[df['album_title'] == 'Jones']

,Unnamed: 0,album_title,artist,release_date,score
681,682,Jones,Bill Ryder,"April 8, 2013",52


In [102]:
for album in df['album_title']:
    print(album)

I See a Darkness
In the Aeroplane Over the Sea
The Hissing of Summer Lawns
Blood on the Tracks
Court and Spark
Pink Moon
Blue
After the Gold Rush
Bryter Layter
I Never Even Asked for Light
Songs of Leonard Cohen
Bon Iver, Bon Iver
On The Beach
Five Leaves Left
Ys
Bridge Over Troubled Water
Carrie & Lowell
The Bootleg Series Vol. 7: No Direction Home: the Soundtrack
Unhalfbricking
Benji
Have One on Me
Illinois
The Glow Pt. 2
Shields
The Bootleg Series Vol. 9: The Witmark Demos: 1962
The Bootleg Series Vol. 6: Bob Dylan Live 1964
Bad Timing
For the Roses
A Crow Looked at Me
Veckatimest
Fleet Foxes
Songs for a Blue Guitar
Red House Painters (Rollercoaster)
Torch of the Mystics
In My Own Time
Just Another Diamond Day
Sweetheart of the Rodeo
Sung Tongs
You Are Free
Central Reservation
Bunny Gets Paid
Chelsea Girl
I Love You, Honeybear
Lost in the Dream
Muchacho
Helplessness Blues
Either/Or
Songs from a Room
What We Did on Our Holidays
Crack
The Bootleg Series Vol. 10: Another Self Portrait 

In [79]:
songs = pd.read_csv('songs.psv', sep='|', header=None)

songs1 = pd.read_csv('songs1.psv', sep='|', header=None)
songs2 = pd.read_csv('songs2.psv', sep='|', header=None, error_bad_lines=False)
songs3 = pd.read_csv('songs3.psv', sep='|', header=None, error_bad_lines=False)

songs = pd.concat((songs, songs1))
songs = pd.concat((songs, songs2))
songs = pd.concat((songs, songs3))

songs.columns = ['song_id', 'song_title', 'href', 'track_count', 'release_date', 'album', 'lyrics', 'artist']

songs.set_index('song_id', inplace=True)

In [112]:
songs.head()

,song_title,href,track_count,release_date,album,lyrics,artist
song_id,,,,,,,
0,A Minor Place,https://genius.com/Bonnie-prince-billy-a-minor...,1.0,1999,I See a Darkness,[hook] n I've been to a minor place n And I ca...,"Bonnie ""Prince"" Billy"
1,Nomadic Revery (All Around),https://genius.com/Bonnie-prince-billy-nomadic...,2.0,1999,I See a Darkness,"Today was one where, lost in thought n I reall...","Bonnie ""Prince"" Billy"
2,I See a Darkness,https://genius.com/Bonnie-prince-billy-i-see-a...,3.0,1999,I See a Darkness,Well you're my friend n (That's what you told ...,"Bonnie ""Prince"" Billy"
3,Another Day Full of Dread,https://genius.com/Bonnie-prince-billy-another...,4.0,1999,I See a Darkness,I like to have a good time n Any of my friends...,"Bonnie ""Prince"" Billy"
4,Death to Everyone,https://genius.com/Bonnie-prince-billy-death-t...,5.0,1999,I See a Darkness,"I am here, right here n Where God puts non asu...","Bonnie ""Prince"" Billy"


In [113]:
songs.isnull().sum()

song_title        0
href              0
track_count     249
release_date      0
album             0
lyrics           11
artist            0
dtype: int64

In [114]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7323 entries, 0 to 7276
Data columns (total 7 columns):
song_title      7323 non-null object
href            7323 non-null object
track_count     7074 non-null float64
release_date    7323 non-null object
album           7323 non-null object
lyrics          7312 non-null object
artist          7323 non-null object
dtypes: float64(1), object(6)
memory usage: 457.7+ KB


In [115]:
songs.loc[songs['track_count'].isnull()]

,song_title,href,track_count,release_date,album,lyrics,artist
song_id,,,,,,,
126,"Bon Iver, Bon Iver [Album Art + Tracklist]",https://genius.com/Bon-iver-bon-iver-bon-iver-...,NaN,"June 21, 2011","Bon Iver, Bon Iver",Album Art n Tracklist n 1. Perth n 2. Minnesot...,Bon Iver
163,Feuilles-o - demo,https://genius.com/Simon-and-garfunkel-feuille...,NaN,"January 26, 1970",Bridge Over Troubled Water,"Feuilles oh, sauvez la vie moi, dans mes yeux ...",Simon & Garfunkel
199,Truck Driver (live in Leamington Spa),https://genius.com/Sun-kil-moon-truck-driver-l...,NaN,2014,Benji,My uncle died in a fire on his birthday n Redn...,Sun Kil Moon
200,Micheline (live in Aveiro),https://genius.com/Sun-kil-moon-micheline-live...,NaN,2014,Benji,Micheline used to come to our house and knock ...,Sun Kil Moon
272,Serious (Short Version),https://genius.com/Donna-allen-serious-short-v...,NaN,1986,Bad Timing,"Baby, tell me n Are you serious n Hey, boy, yo...",Donna Allen
273,Serious (Long Version),https://genius.com/Donna-allen-serious-long-ve...,NaN,1986,Bad Timing,"Baby, tell me n Are you serious n Hey, boy, yo...",Donna Allen
297,A Crow Looked At Me [Tracklist + Album Art],https://genius.com/Mount-eerie-a-crow-looked-a...,NaN,"March 24, 2017",A Crow Looked at Me,"1. ""Real Death"" n 2. ""Seaweed"" n 3. ""Ravens"" ...",Mount Eerie
321,Isles,https://genius.com/Fleet-foxes-isles-lyrics,NaN,2008,Fleet Foxes,Everyone stares when you walk in the room n Th...,Fleet Foxes
322,Fleet Foxes [Tracklist + Album Art],https://genius.com/Fleet-foxes-fleet-foxes-tra...,NaN,2008,Fleet Foxes,1. Sun It Rises n 2. White Winter Hymnal n 3. ...,Fleet Foxes


In [116]:
songs.loc[songs['album'] == 'Bon Iver, Bon Iver']

,song_title,href,track_count,release_date,album,lyrics,artist
song_id,,,,,,,
116,Perth,https://genius.com/Bon-iver-perth-lyrics,1.0,"June 21, 2011","Bon Iver, Bon Iver","I’m tearing up, across your face n Move dust t...",Bon Iver
117,"Minnesota, WI",https://genius.com/Bon-iver-minnesota-wi-lyrics,2.0,"June 21, 2011","Bon Iver, Bon Iver","[Verse 1] n Armour let it through, borne the a...",Bon Iver
118,Holocene,https://genius.com/Bon-iver-holocene-lyrics,3.0,"June 21, 2011","Bon Iver, Bon Iver","Someway, baby, it's part of me, apart from me ...",Bon Iver
119,Towers,https://genius.com/Bon-iver-towers-lyrics,4.0,"June 21, 2011","Bon Iver, Bon Iver","For the love, Iʼd fallen on n In the swampy Au...",Bon Iver
120,Michicant,https://genius.com/Bon-iver-michicant-lyrics,5.0,"June 21, 2011","Bon Iver, Bon Iver","I was unafraid, I was a boy, I was a tender ag...",Bon Iver
121,"Hinnom, TX",https://genius.com/Bon-iver-hinnom-tx-lyrics,6.0,"June 21, 2011","Bon Iver, Bon Iver",Fall in n Fall out n Fall along n In the first...,Bon Iver
122,Wash.,https://genius.com/Bon-iver-wash-lyrics,7.0,"June 21, 2011","Bon Iver, Bon Iver",Climb n Is all we know n When thaw n Is not be...,Bon Iver
123,Calgary,https://genius.com/Bon-iver-calgary-lyrics,8.0,"June 21, 2011","Bon Iver, Bon Iver",[Note] n Don't you cherish me to sleep n Never...,Bon Iver
124,"Lisbon, OH",https://genius.com/Bon-iver-lisbon-oh-lyrics,9.0,"June 21, 2011","Bon Iver, Bon Iver",[Instrumental],Bon Iver


In [117]:
songs.loc[songs['album'] == 'Bridge Over Troubled Water']

,song_title,href,track_count,release_date,album,lyrics,artist
song_id,,,,,,,
150,Bridge Over Troubled Water,https://genius.com/Simon-and-garfunkel-bridge-...,1.0,"January 26, 1970",Bridge Over Troubled Water,[Verse 1] n When you're weary n Feeling small ...,Simon & Garfunkel
151,El Condor Pasa (If I Could) (Ft. Los Incas),https://genius.com/Simon-and-garfunkel-el-cond...,2.0,"January 26, 1970",Bridge Over Troubled Water,[Verse 1: Paul Simon] n I'd rather be a sparro...,Simon & Garfunkel
152,Cecilia,https://genius.com/Simon-and-garfunkel-cecilia...,3.0,"January 26, 1970",Bridge Over Troubled Water,"[Chorus] n 'Cilia, you're breaking my heart n ...",Simon & Garfunkel
153,Keep the Customer Satisfied,https://genius.com/Simon-and-garfunkel-keep-th...,4.0,"January 26, 1970",Bridge Over Troubled Water,"[Verse 1] n Gee, but it’s great to be back hom...",Simon & Garfunkel
154,"So Long, Frank Lloyd Wright",https://genius.com/Simon-and-garfunkel-so-long...,5.0,"January 26, 1970",Bridge Over Troubled Water,"[Verse 1] n So long, Frank Lloyd Wright n I ca...",Simon & Garfunkel
155,The Boxer,https://genius.com/Simon-and-garfunkel-the-box...,6.0,"January 26, 1970",Bridge Over Troubled Water,"[Produced by Roy Halee, Paul Simon, Art Garfun...",Simon & Garfunkel
156,Baby Driver,https://genius.com/Simon-and-garfunkel-baby-dr...,7.0,"January 26, 1970",Bridge Over Troubled Water,[Verse 1] n My daddy was the family bassman n ...,Simon & Garfunkel
157,The Only Living Boy in New York,https://genius.com/Simon-and-garfunkel-the-onl...,8.0,"January 26, 1970",Bridge Over Troubled Water,"[Verse 1] n Tom, get your plane right on time ...",Simon & Garfunkel
158,Why Don't You Write Me,https://genius.com/Simon-and-garfunkel-why-don...,9.0,"January 26, 1970",Bridge Over Troubled Water,[Verse 1] n Why don't you write me n I'm out i...,Simon & Garfunkel


In [119]:
songs.loc[songs['album'] == 'Benji'] # represented on album

,song_title,href,track_count,release_date,album,lyrics,artist
song_id,,,,,,,
188,Carissa,https://genius.com/Sun-kil-moon-carissa-lyrics,1.0,2014,Benji,"Oh Carissa, when I first saw you, you were a l...",Sun Kil Moon
189,I Can't Live Without My Mother's Love,https://genius.com/Sun-kil-moon-i-cant-live-wi...,2.0,2014,Benji,I can live with the sky falling out from above...,Sun Kil Moon
190,Truck Driver,https://genius.com/Sun-kil-moon-truck-driver-l...,3.0,2014,Benji,My uncle died in a fire on his birthday n Redn...,Sun Kil Moon
191,Dogs,https://genius.com/Sun-kil-moon-dogs-lyrics,4.0,2014,Benji,Katy Kerlan was my first kiss n I was only fiv...,Sun Kil Moon
192,Pray for Newtown,https://genius.com/Sun-kil-moon-pray-for-newto...,5.0,2014,Benji,I was a Junior in high school when I turned th...,Sun Kil Moon
193,Jim Wise,https://genius.com/Sun-kil-moon-jim-wise-lyrics,6.0,2014,Benji,"Spent the day with my dad and his old friend, ...",Sun Kil Moon
194,I Love My Dad,https://genius.com/Sun-kil-moon-i-love-my-dad-...,7.0,2014,Benji,When I was young my father taught me not to gl...,Sun Kil Moon
195,I Watched the Film the Song Remains the Same,https://genius.com/Sun-kil-moon-i-watched-the-...,8.0,2014,Benji,I watched the film The Song Remains the Same n...,Sun Kil Moon
196,Richard Ramirez Died Today of Natural Causes,https://genius.com/Sun-kil-moon-richard-ramire...,9.0,2014,Benji,Richard Ramirez died today of natural causes n...,Sun Kil Moon


In [122]:
songs.loc[songs['album'] == 'Trial & Error']

,song_title,href,track_count,release_date,album,lyrics,artist
song_id,,,,,,,
7133,Gossip,https://genius.com/Classified-gossip-lyrics,2.0,2003,Trial & Error,[Verse 1] n Walk in to a room with my hands on...,Classified
7134,Heavy Artillery,https://genius.com/Classified-heavy-artillery-...,3.0,2003,Trial & Error,"[Scratches] n ""I live for the moment, Class go...",Classified
7135,On tha Brink,https://genius.com/Classified-on-tha-brink-lyrics,4.0,2003,Trial & Error,[Hook (x3)] n I'm A Very Bad Daddy I Am I Am n...,Classified
7136,Unexpected (Ft. Mic Boyd),https://genius.com/Classified-unexpected-lyrics,5.0,2003,Trial & Error,[Verse 1] n Now I'm the type of kid who wears ...,Classified
7137,The Way it Is (Ft. DL Incognito & Maestro Fres...,https://genius.com/Classified-the-way-it-is-ly...,6.0,2003,Trial & Error,I played these strings as a kid n I play these...,Classified
7138,Got Luv,https://genius.com/Classified-got-luv-lyrics,7.0,2003,Trial & Error,"[Verse 1:] n Yeah, Yo... Listen n Now I've Got...",Classified
7139,Trial & Error,https://genius.com/Classified-trial-and-error-...,9.0,2003,Trial & Error,I keep on steppin' with class (with class) n N...,Classified
7140,It's Sickening,https://genius.com/Classified-its-sickening-ly...,10.0,2003,Trial & Error,"Oh another one of those songs huh? n Yeah, Bea...",Classified
7141,This is For,https://genius.com/Classified-this-is-for-lyrics,11.0,2003,Trial & Error,"(for the hip hop heads) n (for the MC's, for t...",Classified


In [274]:
folk_musicians = df['artist'].unique()

In [275]:
len(folk_musicians)

378

In [208]:
folk_musicians

{'(Sandy) Alex G',
 'A Hawk and a Hacksaw',
 'A.A. Bondy',
 'Adam Green',
 'Adam Green & Binki Shapiro',
 'Adam Torres',
 'Aimee Mann',
 'Akron/Family',
 'Alasdair Roberts',
 'Alasdair Roberts & Friends',
 'Alela Diane',
 'Alexander',
 'Amandine',
 'Amen Dunes',
 'American Music Club',
 'Anamai',
 'Anaïs Mitchell',
 'Anaïs Mitchell & Jefferson Hamer',
 'Andrew Bird',
 'Ane Brun',
 'Angel Olsen',
 'Angels of Light',
 'Animal Collective',
 'Anni Rossi',
 'Arbouretum',
 'Ariel Pink & Weyes Blood',
 'Augie March',
 'Avey Tare',
 'Avey Tare & Kría Brekkan',
 'Avi Buffalo',
 'Baby Dee',
 'Balmorhea',
 'Basia Bulat',
 'Beck',
 'Bedouine',
 'Beirut',
 'Ben Harper',
 'Ben Harper & The Innocent Criminals',
 'Beth Gibbons & Rustin Man',
 'Beth Orton',
 'Bill Callahan',
 'Bill Fay',
 'Bill Ryder',
 'Billy Bragg & Wilco',
 'Black Prairie',
 'Blitzen Trapper',
 'Blue Roses',
 'Blue Water White Death',
 'Bob Dylan',
 'Bob Weir',
 'Bobb Trimble',
 'Bon Iver',
 "Bonnie 'Prince' Billy",
 "Bonnie 'Prince

In [209]:
df.loc[df['artist'] == 'Sufjan Stevens']

,Unnamed: 0,album_title,artist,release_date,score
16,17,Carrie & Lowell,Sufjan Stevens,"March 31, 2015",93
21,22,Illinois,Sufjan Stevens,"July 5, 2005",92
86,87,Greetings From Michigan: The Great Lakes State,Sufjan Stevens,"July 1, 2003",85
164,165,Seven Swans,Sufjan Stevens,"March 16, 2004",81
275,276,Carrie & Lowell Live,Sufjan Stevens,"April 28, 2017",77
323,324,All Delighted People,Sufjan Stevens,"August 20, 2010",76
431,432,"The Greatest Gift Mixtape – Outtakes, Remixes ...",Sufjan Stevens,"November 24, 2017",72
454,455,The Avalanche,Sufjan Stevens,"July 11, 2006",72


In [260]:
folk_songs = songs.copy()

In [266]:
def is_folk(cell):
    if cell in folk_musicians:
        return 1
    
    elif cell in [
        'The Tallest Man on Earth',
        'Songs Of Green Pheasant',
        'Josh Tillman',
        'Billy Bragg and Wilco',
        'Iron & Wine & Ben Bridwell',
        'Grizzly Bear & Feist',
        'The National',
        'Calexico & Iron & Wine',
        'A Hawk And A Hacksaw',
        'Edward Sharpe and the Magnetic Zeros',
        'The Head and the Heart',
        'Gregory and the Hawk',
        'Noah and the Whale',
        'Flo morrissey',
        'Flo Morrisey',
        'Elvis Perkins In Dearland',
        'Hope Sandoval & the Warm Inventions',
        'Duke Garwood & Mark Lanegan',
        'Barclay James Harvest',
        'Carly Simon',
        'Carly Simon and James Taylor',
        'Dan Bern',
        'Bonnie "Prince" Billy',
    ]:
        return 1
    
    else:
        return 0

In [267]:
folk_songs['is_folk'] = folk_songs['artist'].apply(is_folk)

In [268]:
folk_songs.loc[folk_songs['is_folk'] == 1]

,song_title,href,track_count,release_date,album,lyrics,artist,is_folk
song_id,,,,,,,,
0,A Minor Place,https://genius.com/Bonnie-prince-billy-a-minor...,1.0,1999,I See a Darkness,[hook] n I've been to a minor place n And I ca...,"Bonnie ""Prince"" Billy",1
1,Nomadic Revery (All Around),https://genius.com/Bonnie-prince-billy-nomadic...,2.0,1999,I See a Darkness,"Today was one where, lost in thought n I reall...","Bonnie ""Prince"" Billy",1
2,I See a Darkness,https://genius.com/Bonnie-prince-billy-i-see-a...,3.0,1999,I See a Darkness,Well you're my friend n (That's what you told ...,"Bonnie ""Prince"" Billy",1
3,Another Day Full of Dread,https://genius.com/Bonnie-prince-billy-another...,4.0,1999,I See a Darkness,I like to have a good time n Any of my friends...,"Bonnie ""Prince"" Billy",1
4,Death to Everyone,https://genius.com/Bonnie-prince-billy-death-t...,5.0,1999,I See a Darkness,"I am here, right here n Where God puts non asu...","Bonnie ""Prince"" Billy",1
5,Knockturne,https://genius.com/Bonnie-prince-billy-knocktu...,6.0,1999,I See a Darkness,Fire-burned and blew out flowers n Showing me ...,"Bonnie ""Prince"" Billy",1
6,Madeleine Mary,https://genius.com/Bonnie-prince-billy-madelei...,7.0,1999,I See a Darkness,Sing a song of madeleine-mary n A tune that al...,"Bonnie ""Prince"" Billy",1
7,Song for the New Breed,https://genius.com/Bonnie-prince-billy-song-fo...,8.0,1999,I See a Darkness,Your little feet n Your sharp teeth n The way ...,"Bonnie ""Prince"" Billy",1
8,Today I Was an Evil One,https://genius.com/Bonnie-prince-billy-today-i...,9.0,1999,I See a Darkness,Tonight my eyes were hurting much n As I had s...,"Bonnie ""Prince"" Billy",1


In [269]:
non_folk_songs = folk_songs.loc[folk_songs['is_folk'] == 0]

In [270]:
folk_songs

,song_title,href,track_count,release_date,album,lyrics,artist,is_folk
song_id,,,,,,,,
0,A Minor Place,https://genius.com/Bonnie-prince-billy-a-minor...,1.0,1999,I See a Darkness,[hook] n I've been to a minor place n And I ca...,"Bonnie ""Prince"" Billy",1
1,Nomadic Revery (All Around),https://genius.com/Bonnie-prince-billy-nomadic...,2.0,1999,I See a Darkness,"Today was one where, lost in thought n I reall...","Bonnie ""Prince"" Billy",1
2,I See a Darkness,https://genius.com/Bonnie-prince-billy-i-see-a...,3.0,1999,I See a Darkness,Well you're my friend n (That's what you told ...,"Bonnie ""Prince"" Billy",1
3,Another Day Full of Dread,https://genius.com/Bonnie-prince-billy-another...,4.0,1999,I See a Darkness,I like to have a good time n Any of my friends...,"Bonnie ""Prince"" Billy",1
4,Death to Everyone,https://genius.com/Bonnie-prince-billy-death-t...,5.0,1999,I See a Darkness,"I am here, right here n Where God puts non asu...","Bonnie ""Prince"" Billy",1
5,Knockturne,https://genius.com/Bonnie-prince-billy-knocktu...,6.0,1999,I See a Darkness,Fire-burned and blew out flowers n Showing me ...,"Bonnie ""Prince"" Billy",1
6,Madeleine Mary,https://genius.com/Bonnie-prince-billy-madelei...,7.0,1999,I See a Darkness,Sing a song of madeleine-mary n A tune that al...,"Bonnie ""Prince"" Billy",1
7,Song for the New Breed,https://genius.com/Bonnie-prince-billy-song-fo...,8.0,1999,I See a Darkness,Your little feet n Your sharp teeth n The way ...,"Bonnie ""Prince"" Billy",1
8,Today I Was an Evil One,https://genius.com/Bonnie-prince-billy-today-i...,9.0,1999,I See a Darkness,Tonight my eyes were hurting much n As I had s...,"Bonnie ""Prince"" Billy",1


In [273]:
for artist in non_folk_songs['artist'].unique():
    print(artist)

Troye Sivan
Donna Allen
Chacho Brodas
RVNHD
EarthGang
Circuit des Yeux
Mike Shinoda
Brightblack morning light
Trailerpark
Dirty Projectors
The Books
Yeasayer
My Brightest Diamond
Kronos Quartet
Antony with Bryce Dessner
Justin Vernon & Aaron Dessner
Spoon
Arcade Fire
My Morning Jacket
Sharon Jones & The Dap-Kings
David Andrew Sitek
Buck 65
The New Pornographers
Yo La Tengo
Stuart Murdoch
Jónsi & Alex
Cat Power & Dirty Delta Blues
Conor Oberst & Gillian Welch
Blonde Redhead & Devastations
Kevin Drew
Beach House
Lupe Fiasco
Elton John
Wyclef Jean
PUNPEE
Anathema
Sonny Landreth
Case/lang/veirs
Euro
Joey Bada$$
Hefna Gwap
Rokamouth
CJ Fly
Kirk Knight
Skizzy Mars
Ski Mask The Slump God
Rise Against
Kendrick Lamar
Matthew Friedberger
Joan Shelly
Reem Riches
TeeCee4800
Ty Dolla $ign
Paramore
Indica
The Swimmers
Rihanna
Moira Dela Torre
Bas
SURAN
A$AP Ferg
Mark Lanegan
Hiroshima
Marshall Marshall
Shy Fiction and JBRAGG
​The Amazing
KILLSTREAK
Travis Scott
​blackbear
Bastille
Capyac
Nico & Vinz

In [277]:
folk_songs.loc[folk_songs['is_folk'] == 0]

,song_title,href,track_count,release_date,album,lyrics,artist,is_folk
song_id,,,,,,,,
64,WILD (Solo),https://genius.com/Troye-sivan-wild-solo-lyrics,1.0,"December 4, 2015",Blue Neighbourhood,[Verse 1] n Trying hard not to fall n On the w...,Troye Sivan,0
65,BITE,https://genius.com/Troye-sivan-bite-lyrics,2.0,"December 4, 2015",Blue Neighbourhood,[Verse 1] n Kiss me on the mouth and set me fr...,Troye Sivan,0
66,FOOLS,https://genius.com/Troye-sivan-fools-lyrics,3.0,"December 4, 2015",Blue Neighbourhood,[Verse 1-Troye Sivan] n I am tired of this pla...,Troye Sivan,0
67,EASE (Ft. Broods),https://genius.com/Troye-sivan-ease-lyrics,4.0,"December 4, 2015",Blue Neighbourhood,[Verse 1: Troye Sivan] n I'm down to my skin a...,Troye Sivan,0
68,THE QUIET,https://genius.com/Troye-sivan-the-quiet-lyrics,5.0,"December 4, 2015",Blue Neighbourhood,"[Verse 1: Troye Sivan] n Growing distance, fre...",Troye Sivan,0
69,DKLA (Ft. Tkay Maidza),https://genius.com/Troye-sivan-dkla-lyrics,6.0,"December 4, 2015",Blue Neighbourhood,[Intro: Troye Sivan] n Wrapped my thoughts aro...,Troye Sivan,0
70,TALK ME DOWN,https://genius.com/Troye-sivan-talk-me-down-ly...,7.0,"December 4, 2015",Blue Neighbourhood,[Verse 1] n I wanna sleep next to you n But th...,Troye Sivan,0
71,COOL,https://genius.com/Troye-sivan-cool-lyrics,8.0,"December 4, 2015",Blue Neighbourhood,[Verse 1] n Sweet life living n Pools and swim...,Troye Sivan,0
72,HEAVEN (Ft. Betty Who),https://genius.com/Troye-sivan-heaven-lyrics,9.0,"December 4, 2015",Blue Neighbourhood,[Verse 1: Troye Sivan] n The truth runs wild n...,Troye Sivan,0


In [279]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [283]:
stop_words = ENGLISH_STOP_WORDS

In [288]:
Unfreeze(stop_words)

NameError: name 'Unfreeze' is not defined